# Introduction to scikit-learn
Author: `chloe-agathe.azencott@mines-paristech.fr` based on materials from [Alexandre Gramfort](http://alexandre.gramfort.net/) and [Jake Vanderplas](https://github.com/jakevdp) 

The goal of this notebook is to learn how to use scikit-learn to solve machine learning problems, and how to evaluate and compare models. We will work with the same Digits data set as in Notebook 3.

## Load the data

In [ ]:
import numpy as np
%pylab inline

In [ ]:
# Load data
from sklearn.datasets import load_digits 

digits = load_digits()

# Get descriptors and target to predict
X, y = digits.data, digits.target

# Get the shape of the data
print("Number of samples: %d" % X.shape[0])
print("Number of pixels: %d" % X.shape[1])
print("Number of classes: %d" % len(np.unique(y))) # number of unique values in y

## 1. Dimensionality reduction

Each object (image) in this data set is represented using 64 features. This makes plotting all the objects on the same figure difficult. Let us use PCA to project the images on 2 dimensions and visualize them.  

In [ ]:
from sklearn import decomposition

In [ ]:
# Create a pca object
pca = decomposition.PCA(n_components=2)

# Apply to the data
X_proj = pca.fit_transform(X)
print(X_proj.shape)

In [ ]:
# Visualize the projected data
plt.scatter(X_proj[:, 0], # first dimension
            X_proj[:, 1], # second dimension
            c=y, # color by label
            edgecolor='none', # remove dot border
            alpha=0.5 # use transparency to better see overlapping dots
           ) 
plt.colorbar(label='digit label', ticks=range(10))

__Question__ What do you observe? Do you think the classification problem will be easy or hard?

A popular alternative to PCA, which projects the data linearly, is the [tSNE](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) approach. It is very similar to run in scikit-learn:

In [ ]:
from sklearn import manifold

In [ ]:
# Create a tSNE object
tsne = manifold.TSNE(n_components=2)

# Apply to the data
X_proj = tsne.fit_transform(X)
print(X_proj.shape)

In [ ]:
# Visualize the projected data
plt.scatter(X_proj[:, 0], # first dimension
            X_proj[:, 1], # second dimension
            c=y, # color by label
            edgecolor='none', # remove dot border
            alpha=0.5 # use transparency to better see overlapping dots
           ) 
plt.colorbar(label='digit label', ticks=range(10))

__Question__ What do you observe? Would you rather use the tSNE features or the PCA features for learning?

## 2. Clustering 
K-means is an algorithm for unsupervised clustering: that is, finding clusters (or groups) in data based on the data attributes alone (not the labels). K-means groups samples based on distances: points get assigned to the cluster to which they are the closest (based on the distance to the centroid), centroids are recomputed, and the procedure is iterated until convergence.

In k-means, the number of clusters k is a hyperparameter the user must provide.

Let us look at how it performs on the Digits data.

In [ ]:
from sklearn import cluster

In [ ]:
# create a k-means clustering object, for k=10
clustering = cluster.KMeans(10)

# apply it to the data
clusters = clustering.fit_predict(X)

Let us now look at the cluster centroids (i.e. the mean point of each cluster):

In [ ]:
fig = plt.figure(figsize=(8, 3))
for i in range(10):
    ax = fig.add_subplot(2, 5, 1 + i, xticks=[], yticks=[])
    ax.imshow(clustering.cluster_centers_[i].reshape((8, 8)), cmap=plt.cm.binary)

Even without the labels, the clustering algorithm has identified reasonable representants for each class!

Let us compare the clusters we obtained to the true labels. First, notice that the cluster centroids do not appear in the same order as the true labels (the first one is not necessarily a 1), because an unsupervised algorithm has no means to know what label we would like to assign to a cluster it found. We will start by reassigning cluster labels to match the most common digit in the cluster.

In [ ]:
from scipy.stats import mode

pred_labels = np.zeros_like(clusters)
for i in range(10):
    mask = (clusters == i)
    pred_labels[mask] = mode(digits.target[mask])[0]

To compare these clusters to the true labels, we can look at their accuracy

In [ ]:
from sklearn import metrics

In [ ]:
print("Accuracy: %.2f" % metrics.accuracy_score(y, pred_labels))

And look at the confusion matrix

In [ ]:
print(metrics.confusion_matrix(y, pred_labels))

In [ ]:
# Visualization
plt.imshow(confusion_matrix(y, pred_labels),
           cmap='Blues', interpolation='nearest')
plt.colorbar()
plt.grid(False)
plt.ylabel('true')
plt.xlabel('predicted')

__Question__ Which class was the most difficult to identify for the clustering algorithm? Does this match what you observed when looking at the centroids?

We can also visualize the result of the clustering algorithm using the 2 tSNE dimensions.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(9, 4))
# Plot 1
ax[0].scatter(X_proj[:, 0], X_proj[:, 1], c=pred_labels, 
             edgecolor='none', alpha=0.5)
ax[0].set_title('learned cluster labels')

# Plot 2
t = ax[1].scatter(X_proj[:, 0], X_proj[:, 1], c=y, 
             edgecolor='none', alpha=0.5)
ax[1].set_title('true labels')
plt.colorbar(t, label='digit label', ticks=range(10))

To get more distinct colors, you can use a different colormap:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(9, 4))
# Plot 1
ax[0].scatter(X_proj[:, 0], X_proj[:, 1], c=pred_labels, 
             edgecolor='none', alpha=0.5,
             cmap = plt.cm.get_cmap('rainbow', 10))
ax[0].set_title('learned cluster labels')

# Plot 2
t = ax[1].scatter(X_proj[:, 0], X_proj[:, 1], c=y, 
             edgecolor='none', alpha=0.5,
             cmap = plt.cm.get_cmap('rainbow', 10))
ax[1].set_title('true labels')
plt.colorbar(t, label='digit label', ticks=range(10))

__Question__ Which digits are easy to predict? Which ones are confused with which other digits? Does this match the confusion matrix?

__Question__ Reproduce this analysis using a different number of clusters (e.g. k=7) 

## 3. Classification
So, how well can we learn to separate these images using labeled data?

### 3.1 Model validation and selection
An very important part of machine learning is __model validation__: that is, determining how well your model will generalize from the training data to future unlabeled data. Using a simple algorithm that returns the stored label for any observation from the data set would result in very good predictions, but would not have _learned_ anything, and would be rubbish at predicting the label of a new handwritten digit.

We will therefore systematically split our data in a _training set_, on which we'll use the labels to _learn_, or _train_ the model, and a _test set_, on which we'll only use the labels to evaluate the model's ability to make correct predictions. __Not using test data at train time is absolutely essential to estimating generalization ability.__

In [ ]:
from sklearn import model_selection 
Xtrain, Xtest, ytrain, ytest = model_selection.train_test_split(X, y, random_state=42)

print(Xtrain.shape, Xtest.shape)

Using a fixed random seed (`random_state`) will ensure you can reproduce your experiments, i.e. always have the same samples in your train (resp. test) set. Someone running the experiments with the same seed on a different computer will have a different train/test split, though.

__Question__ What fraction of our data will we use for training? For testing?

Suppose we want to choose between different machine learning algorithms, or between different values of the hyperparameters of the same algorithm. Of course, we can pick the model with the best performance on the test set. But how will we evaulate its performance?

__Question__ Why isn't the performance of the best-performing algorithm on the test set a good estimation of the generalization performance (i.e. performance on new data)?

In this situation, it is common to use __cross-validation__ on the train set to select the best algorithm/model. The `sklearn.model_selection` module provides cross-validation facilities. In addition, many supervised learning algorithms of scikit-learn can be used with such an internal cross-validation procedure to select optimal hyperparameter(s).

## 3.2 Logistic regression 

[LogisticRegression documentation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

[LogisticRegressionCV documentation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html)

The logistic regression implementation of scikit-learn minimizes C x logistic empirical error + regularization. Hence the non-regularized logistic regression is obtained when C is very large.

In [ ]:
from sklearn import linear_model 

# Create a logistic regression classifier
clf = linear_model.LogisticRegression(penalty='l2', 
                                      C=1e8) # large value of C = no regularization

# Fit the classifier on train data
clf.fit(Xtrain, ytrain)

# Apply the classifier to test data
ypred = clf.predict(Xtest)

__Question__ What is the accuracy of this classifier? Display its confusion matrix.

Can regularization improve this performance? Let us look at what happens when we change this parameter. We will use a cross-validation on the training set. 

In [ ]:
# Define a range of values for C
c_values = np.logspace(-5, 5, num=11)
print c_values

accuracies = [] # where to store mean accuracy for each value of C
for c_val in c_values:
    # create a logistic regression classifier
    clf = linear_model.LogisticRegression(penalty='l2', C=c_val)
    
    # compute the 5-fold cross validation score of this classifier
    scores = model_selection.cross_val_score(clf, Xtrain, ytrain, 
                                             cv=5,
                                             scoring='accuracy')
    accuracies.append(np.mean(scores))

In [ ]:
plt.semilogx(c_values, accuracies, marker='o') # plot using a log-scale on the x-axis
plt.xlabel('Mean cross-validated accuracy')
plt.ylabel('Value of C')

__Question__ What is the impact of having a very small C? Why?

Scikit-learn lets us tune the parameter C automatically, by cross-validation on the training set.

In [ ]:
clf = linear_model.LogisticRegressionCV(Cs=10, 
                                        penalty='l2')
clf.fit(Xtrain, ytrain)
ypred = clf.predict(Xtest)

__Question__ What values of C were tested? What is the accuracy now?

## 3.2 Nearest neighbors

[KNeighborsClassifier documentation](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier)

The nearest neighbor algorithm classifies an observation according to the labels of the k nearest points in the training set.

In [ ]:
from sklearn import neighbors

In [ ]:
# Create a kNN classifier
clf = neighbors.KNeighborsClassifier(n_neighbors=3)

# Fit the classifier on train data
clf.fit(Xtrain, ytrain)

# Apply the classifier to test data
ypred = clf.predict(Xtest)

__Question__ What is the accuracy of this classifier? Display its confusion matrix. How does it compare to the logistic regression?

Again, we have a hyperparameter to fit: the number of nearest neighbors.

Let us see how it impacts cross-validates performance on the training set.

In [ ]:
# Define a range of values for k
k_values = np.array(np.linspace(3, 23, 10), dtype=int)
print(k_values)

In [ ]:
accuracies = [] # where to store mean accuracy for each value of k
for k_val in k_values:
    # create a kNN classifier
    clf = neighbors.KNeighborsClassifier(n_neighbors=k_val)
    
    # compute the 5-fold cross validation score of this classifier
    scores = model_selection.cross_val_score(clf, Xtrain, ytrain, 
                                             cv=10,
                                             scoring='accuracy')
    accuracies.append(np.mean(scores))

In [ ]:
plt.plot(k_values, accuracies, marker='o') 
plt.xlabel('Mean cross-validated accuracy')
plt.ylabel('Value of k')

__Question__ What is the optimal number of nearest neighbors to use for this data?

__Question__ Train a kNN that uses this optimal number of neighbors on the train set and evaluate it on the test set.

## 3.3 SVM
### 3.3.1 Linear SVM

[LinearSVC documentation](http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html)

Let us start with a linear SVM.

In [ ]:
from sklearn import svm

In [ ]:
# Create a linear SVM classifier
clf = svm.LinearSVC(dual=False) # when number of samples > number of features, 
                                # solving the primal is more efficient.

# Fit the classifier on train data
clf.fit(Xtrain, ytrain)

# Apply the classifier to test data
ypred = clf.predict(Xtest)

In [ ]:
print("Accuracy: %.3f" % metrics.accuracy_score(ytest, ypred))

__Question__ What is the accuracy of this classifier? Display its confusion matrix. What was the value of the C parameter?

We will now use a grid search to optimize the parameter C.

[GridSearchCV documentation](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

In [ ]:
# define a grid (here, a line) of parameters to explore
param_grid = {'C': np.logspace(-5, 5, 11)}

# define the base classifier
clf = svm.LinearSVC(dual=False)

# define the grid search
gs = model_selection.GridSearchCV(clf, param_grid, cv=5, 
                                  scoring='accuracy')
# train
gs.fit(Xtrain, ytrain)

# print the best score and corresponding parameter
print("Best accuracy: %.3f" % gs.best_score_)
print("Optimal parameter:", gs.best_params_)

Information regarding the cross-validation experiment are stored in `gs.cv_results_`. 

In particular, we can observe how the mean cross-validated accuracy varies with C.

In [ ]:
plt.semilogx(np.array(gs.cv_results_['param_C'], dtype='float'), 
             gs.cv_results_['mean_test_score'], marker='o') # plot using a log-scale on the x-axis
plt.xlabel('Mean cross-validated accuracy')
plt.ylabel('Value of C')

We can now train an SVM with optimal C parameter on the train set and evaluate it on the test set.

In [ ]:
# Create a linear SVM classifier
clf = svm.LinearSVC(dual=False, C=gs.best_params_['C'])

# Fit the classifier on train data
clf.fit(Xtrain, ytrain)

# Apply the classifier to test data
ypred = clf.predict(Xtest)

__Question__ How does this classifier's accuracy compare to that of the logistic regression? Of the nearest neighbors? Look at the confusion matrix: is it always the same digits that are difficult to predict?

### 3.3.2 Kernel SVM

Although our linear SVM already performs well, the power of SVMs typically come from their ability to solve the problem in a non-linear way, thanks to a kernel.

[SVC documentation](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)


In [ ]:
# Create an RBF SVM classifier
clf = svm.SVC(kernel='rbf') 

# Fit the classifier on train data
clf.fit(Xtrain, ytrain)

# Apply the classifier to test data
ypred = clf.predict(Xtest)

__Question__ What is the accuracy of this classifier? 

__Question__ Choose two parameters to optimize to try and improve this performance.

__Question__ Train the model with optimal parameters on the train set and evaluate it on the test set. Compare it to the previous models.

## 3.4 Random forests

[RandomForestClassifier documentation](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

Now you can try applying random forests to the Digits data. Refer to the documentation to create and evaluate a random forest classifier with default parameter, as well as to choose which parameters to optimize.